## PREPARE ENV


In [1]:
! pip install numpy pandas matplotlib seaborn nltk scikit-learn
! pip install textstat wordcloud imbalanced-learn sentence-transformers
! pip install ipywidgets

#install Natural Language Toolkit [NLTK] data packages
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/203.1 MB 1.2 MB/s eta 0:02:49
   ---------------------------------------- 0.8/203.1 MB 1.3 MB/s eta 0:02:31
   ---------------------------------------- 1.0/203.1 MB 1.3 MB/s eta 0:02:37
   ---------------------------------------- 1.3/203.1 MB 1.3 MB/s eta 0:02:31
   ---------------------------------------- 1.6/203.1 MB 1.4 MB/s eta 0:02:29
   ---------------------------------------- 1.8/203.1 MB 1.4 MB/s eta 0:02:26
   ---------------------------------------- 2.4/203.1 MB 1.4 MB/s eta 0:02:26
    --------------------------------------- 2.6/203.1 MB 1.4 MB/s eta 0:02:25
    --------------------------------------- 2.9/203.1 MB 1.4 MB/s eta 0:02:27
    --------------------------------------- 3.1/203.1 MB 1.4 MB/s eta 0:02:26
    --------------------------------------- 3.4/203.1 MB 1.4 MB/s eta 0:02:24



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jangl\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jangl\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jangl\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jangl\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [2]:
# ENABLE GPU FOR SKLEARN
# Intel® Extension for Scikit-learn

! pip install scikit-learn-intelex

from sklearnex import patch_sklearn
patch_sklearn()


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
   ------- -------------------------------- 0.5/3.0 MB 1.7 MB/s eta 0:00:02
   ---------- ----------------------------- 0.8/3.0 MB 1.7 MB/s eta 0:00:02
   ----------------- ---------------------- 1.3/3.0 MB 1.6 MB/s eta 0:00:02
   ----------------- ---------------------- 1.3/3.0 MB 1.6 MB/s eta 0:00:02
   ------------------------ --------------- 1.8/3.0 MB 1.5 MB/s eta 0:00:01
   ---------------------------- ----------- 2.1/3.0 MB 1.5 MB/s eta 0:00:01
   ------------------------------- -------- 2.4/3.0 MB 1.5 MB/s eta 0:00:01
   ----------------------------------- ---- 2.6/3.0 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 3.0/3.0 MB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/68.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/68.6 MB ? eta -:--:--
   -----------------------------------

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
from sklearn import neural_network, metrics, ensemble, linear_model, feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from textstat import flesch_reading_ease
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import collections
import sklearn
import scipy
import nltk
import re
import os

## DATA LOADING

In [4]:
sourceFolder = "Challenge Emotion Detection\MDW2024_Docs_Players"

mapping = {
    0: 'anger',
    1: 'joy',
    2: 'optimism',
    3: 'sadness'
}

#train data
trainData              = pd.read_csv(os.path.join(sourceFolder, 'train_text.txt'),   delimiter=r"\n", engine='python', header=None, names=['tweet'])
trainLabels            = pd.read_csv(os.path.join(sourceFolder, 'train_labels.txt'), delimiter=r"\n", engine='python', header=None, names=['labels'])
trainData              = trainData.join(trainLabels)
trainData['sentiment'] = trainData['labels'].map(mapping)

#test data
valData                = pd.read_csv(os.path.join(sourceFolder, 'val_text.txt'),    delimiter=r"\n", engine='python', header=None, names=['tweet'])
valLabels              = pd.read_csv(os.path.join(sourceFolder, 'val_labels.txt'),  delimiter=r"\n", engine='python', header=None, names=['labels'])
valData                = valData.join(valLabels)
valData['sentiment']   = valData['labels'].map(mapping)

#test data
testData               = pd.read_csv(os.path.join(sourceFolder, 'test_text.txt'),   delimiter=r"\n", engine='python', header=None, names=['tweet'])


FileNotFoundError: [Errno 2] No such file or directory: 'Challenge Emotion Detection\\MDW2024_Docs_Players\\train_text.txt'

## PREPROCESSING

In [ ]:
trainData["clean_tweet"] = trainData["tweet"]
valData[  "clean_tweet"] = valData[  "tweet"]
testData[ "clean_tweet"] = testData[ "tweet"]

In [ ]:
# NEGATION (Convert words like "aren't" in "are not")
re_negation = re.compile("n't ") 
def negation_abbreviated_to_standard(tweet):
    tweet = re_negation.sub(" not ", tweet)
    return tweet

#trainData["clean_tweet"] = trainData["clean_tweet"].apply(negation_abbreviated_to_standard)
#valData[  "clean_tweet"] = valData[  "clean_tweet"].apply(negation_abbreviated_to_standard)
#testData[ "clean_tweet"] = testData[ "clean_tweet"].apply(negation_abbreviated_to_standard)

In [ ]:
# ENGLISH SLANG CLEANING
slang_dict = {
    "lol": "laughing out loud",
    "lmao": "laughing my ass off",
    "rofl": "rolling on the floor laughing",
    "brb": "be right back",
    "btw": "by the way",
    "imo": "in my opinion",
    "tbh": "to be honest",
    "ttyl": "talk to you later",
    "afaik": "as far as I know",
    "omg": "oh my god",
    "wtf": "what the f***",
    "smh": "shaking my head",
    "jk": "just kidding",
    "np": "no problem",
    "ty": "thank you",
    "pls": "please",
    "gtg": "got to go",
    "gonna": "going to",
    "wanna": "want to",
    "kinda": "kind of",
    "gotta": "got to",
    "sup": "what's up",
    "yas": "yes",
    "bae": "before anyone else/baby/boyfriend/girlfriend",
    "ily": "I love you",
    "nvm": "nevermind",
    "rn": "right now",
    "cray": "crazy",
    "wyd": "what are you doing?",
    "lmk": "let me know",
    "dm": "direct message",
    "irl": "in real life",
    "rt": "retweet",
    "lit": "exciting",
    "salty": "angry",
    "savage": "cool, impressive",
    "feels": "emotions, feelings",
    "mad": "angry, upset",
    "sad boi hours": "feeling sad and melancholic",
    "blessed": "fortunate, happy",
    "shook": "surprised, shocked",
    "whew": "relief, exhaustion",
    "oof": "ouch, unfortunate",
    "rip": "rest in peace/ruined/gone wrong",
    "dead": "hilarious, extremely funny",
    "triggered": "easily upset or offended",
    "hashtag": "#",
    "mention": "@username",
    "tweetstorm": "a series of connected tweets",
    "stan": "to be an extremely devoted fan",
    "flex": "to show off",
}

def replace_slang(text):
    for slang, standard in slang_dict.items():
        text = text.replace(slang, standard)
    return text

trainData["clean_tweet"] = trainData["clean_tweet"].apply(replace_slang)
valData[  "clean_tweet"] = valData[  "clean_tweet"].apply(replace_slang)
testData[ "clean_tweet"] = testData[ "clean_tweet"].apply(replace_slang)

In [ ]:
# DATA CLEANING

def dataCleaning(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#(\w+)", r"\1", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = text.lower()
    text = re.sub(r"\bim\b", "", text)
    return text

trainData["clean_tweet"] = trainData["clean_tweet"].apply(dataCleaning)
valData[  "clean_tweet"] = valData[  "clean_tweet"].apply(dataCleaning)
testData[ "clean_tweet"] = testData[ "clean_tweet"].apply(dataCleaning)

In [ ]:
#TOKENIZATION
stop_words = set(nltk.corpus.stopwords.words('english'))
def dataTokenization(text):
    tokens = nltk.tokenize.word_tokenize(text.lower(), language='english') # tokens = text.split()
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words and word != "im"]
    return tokens

trainData["tokens"] = trainData["clean_tweet"].apply(dataTokenization)
valData[  "tokens"] = valData[  "clean_tweet"].apply(dataTokenization)
testData[ "tokens"] = testData[ "clean_tweet"].apply(dataTokenization)

In [ ]:
def dataLemmization(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

def dataStemming(tokens):
    stemmer = nltk.stem.PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

trainData["stems"] = trainData["tokens"].apply(dataStemming)
valData[  "stems"] = valData[  "tokens"].apply(dataStemming)
testData[ "stems"] = testData[ "tokens"].apply(dataStemming)

trainData["tokens"] = trainData["tokens"].apply(dataLemmization)
valData[  "tokens"] = valData[  "tokens"].apply(dataLemmization)
testData[ "tokens"] = testData[ "tokens"].apply(dataLemmization)

In [ ]:
#ADD MEASURES
trainData["nChars"] = trainData["tweet"].apply(len)
valData[  "nChars"] = valData[  "tweet"].apply(len)
testData[ "nChars"] = testData[ "tweet"].apply(len)

trainData["nWords"] = trainData["tokens"].apply(len)
valData[  "nWords"] = valData[  "tokens"].apply(len)
testData[ "nWords"] = testData[ "tokens"].apply(len)

## EXPLORE DATA

In [ ]:
palette = {
    "optimism": '#1f77b4',
    "anger":    '#ff7f0e',
    "joy":      '#2ca02c',
    "sadness":  '#d62728'
}

In [ ]:

plt.figure(figsize=(20,5))

ax = plt.subplot(1,3,1)
sns.boxplot(
    ax=ax,
    data=trainData, 
    x='sentiment', 
    y='nWords', 
    hue='sentiment',
    hue_order=palette.keys(),
    palette=palette.values(),
    legend=False,
    notch=True,
    showcaps=False,
    medianprops={"color": "r", "linewidth": 2},
)
plt.title('Number of words per sentiment')
plt.ylabel('Number of Words')

ax = plt.subplot(1,3,2)
sns.countplot(
    ax=ax,
    data=trainData,
    x='sentiment',
    hue='sentiment',
    hue_order=palette.keys(),
    palette=palette.values(),
)
plt.title('Sentiment distribution')
plt.ylabel('Number of occurrency')

ax = plt.subplot(1,3,3)
sns.boxplot(
    ax=ax,
    data=trainData, 
    x='sentiment', 
    y='nChars', 
    hue='sentiment', 
    hue_order=palette.keys(),
    palette=palette.values(),
    legend=False,
    notch=True,
    showcaps=False,
    medianprops={"color": "r", "linewidth": 2},
)
plt.title('Number of chars per sentiment')
plt.ylabel('Number of Chars')

plt.show()

In [ ]:
most_common_words = {}
top_n = 10   

#COUNT WORD OCCURRENCE PER SENTIMENT
for sentiment in mapping.keys():
    sentiment_tokens = trainData[trainData['labels'] == sentiment]['tokens']
    word_counts = collections.Counter()
    for tokens in sentiment_tokens: word_counts.update(tokens)
    most_common_words[mapping[sentiment]] = word_counts.most_common(10)

#PLOT
plt.figure(figsize=(20,15))
for i, sentiment in enumerate(most_common_words.keys()):
    plt.subplot(2,2,i+1)
    sns.barplot(
        data=pd.DataFrame(most_common_words[sentiment], columns=['word', 'n']), 
        x='n',
        y='word', 
        hue='word', 
        palette=[palette[sentiment]] * top_n
    )
    plt.ylabel(None)
    plt.xlabel(None)
    plt.title(f'Top {top_n} used words in {sentiment.capitalize()}')
    
plt.show()

In [ ]:
most_common_words = {}
nGramLen = 2
top_n = 10   

#COUNT nGrams OCCURRENCE PER SENTIMENT
for sentiment in mapping.keys():
    sentiment_tokens = trainData[trainData['labels'] == sentiment]['tokens']
    word_counts = collections.Counter()
    for tokens in sentiment_tokens: word_counts.update(' '.join(tokens[idx : idx + nGramLen]) for idx in range(len(tokens) - nGramLen + 1))
    most_common_words[mapping[sentiment]] = word_counts.most_common(10)

#PLOT
plt.figure(figsize=(20,15))
for i, sentiment in enumerate(most_common_words.keys()):
    plt.subplot(2,2,i+1)
    sns.barplot(
        data=pd.DataFrame(most_common_words[sentiment], columns=['word', 'n']), 
        x='n',
        y='word', 
        hue='word', 
        palette=[palette[sentiment]] * top_n
    )
    plt.ylabel(None)
    plt.xlabel(None)
    plt.title(f'Top {top_n} used nGram of size {nGramLen} in {sentiment.capitalize()}')
    
plt.show()

In [ ]:
most_common_ngrams = {}
nGramLen = 2
top_n = 10  

tfidf = TfidfVectorizer(ngram_range=(nGramLen, nGramLen))
tfidf_matrix = tfidf.fit_transform(trainData['tokens'].apply(lambda tokens: ' '.join(tokens)))

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
tfidf_df['labels'] = trainData["labels"]

#COUNT nGrams OCCURRENCE PER SENTIMENT with max tf-idf
for sentiment in mapping.keys():    
    group_df = tfidf_df[tfidf_df['labels'] == sentiment].drop(['labels'], axis=1)
    max_tfidf = group_df.max().sort_values(ascending=False).head(top_n)
    most_common_ngrams[mapping[sentiment]] = max_tfidf.reset_index().rename(columns={'index': 'nGram', 0: 'tf_idf'})

#PLOT
plt.figure(figsize=(20,15))
for i, sentiment in enumerate(most_common_words.keys()):
    plt.subplot(2,2,i+1)
    sns.barplot(
        data=most_common_ngrams[sentiment], 
        x='tf_idf',
        y='nGram', 
        hue='nGram', 
        palette=[palette[sentiment]] * top_n
    )
    plt.ylabel(None)
    plt.xlabel(None)
    plt.title(f'Top {top_n} TF-IDF nGram of size {nGramLen} in {sentiment.capitalize()}')
    
plt.show()

In [ ]:
trainData['Flesch'] = trainData['tweet'].apply(flesch_reading_ease)
plt.figure(figsize=(8,6))
sns.kdeplot(
    data=trainData, 
    x='Flesch', 
    hue='sentiment',
    hue_order=palette.keys(),
    palette=palette.values(),
    fill=True,
    common_norm=False,
)
plt.xlabel('Flesch Index')
plt.ylabel('Density')
plt.title('Tweet Readability (Flesh Index)')
plt.show()

## FEATURE EXTRACTION

In [ ]:
# EMBEDDING
embedded_model = SentenceTransformer('all-mpnet-base-v2')
def extractEmbedding(text): return embedded_model.encode(text)

# TF-IDS (1,2 nGrams)
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', max_features=3000)
tfidf_vectorizer.fit(trainData['clean_tweet'])

def extractFeatures(df):
    feat_nWords = np.vstack(df['tokens'].apply(len).values)
    feat_nChars = np.vstack(df['clean_tweet'].apply(len).values)
    feat_flesch = np.vstack(df['clean_tweet'].apply(flesch_reading_ease).values)
    feat_embedd = np.vstack(df['clean_tweet'].apply(extractEmbedding).values)
    feat_embedd = scipy.sparse.csr_matrix(feat_embedd)
    feat_df_idf = tfidf_vectorizer.transform(df['clean_tweet'])
    
    X = scipy.sparse.hstack([
        feat_nWords,
        feat_nChars,
        feat_flesch,
        feat_embedd,
        feat_df_idf,
    ])
    y = None if 'labels' not in df else df['labels']
    return X,y

xTrain, yTrain = extractFeatures(trainData)
xVal,   yVal   = extractFeatures(valData)
xTest,  _      = extractFeatures(testData)



## TRAIN AND PREDICT

In [ ]:
#TEST AND HYPER-TUNE YOUR MODEL...

clf = sklearn.linear_model.LogisticRegression(max_iter=2000)
#clf = sklearn.ensemble.GradientBoostingClassifier(n_estimators=800)
#clf = sklearn.ensemble.RandomForestClassifier(n_estimators=200)
#clf = sklearn.neural_network.MLPClassifier(300)

clf.fit(xTrain, yTrain)

yTrain_pred = clf.predict(xTrain)
yTrain_prob = clf.predict_proba(xTrain)
yVal_pred   = clf.predict(xVal)
yVal_prob   = clf.predict_proba(xVal)

## RESULTS ON TEST

In [ ]:
cm = sklearn.metrics.confusion_matrix(yTrain, yTrain_pred, normalize='true')
disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=mapping.values())
disp.plot()
plt.show()

In [ ]:
print(sklearn.metrics.classification_report(yTrain, yTrain_pred, target_names=mapping.values()))

In [ ]:
f1  = sklearn.metrics.f1_score(     yTrain, yTrain_pred, average="macro")
auc = sklearn.metrics.roc_auc_score(yTrain, yTrain_prob, average="macro", multi_class='ovr')

print('f1-score: ', f1)
print('auc:      ', auc)

## RESULTS ON VALIDATION

In [ ]:
cm = sklearn.metrics.confusion_matrix(yVal, yVal_pred, normalize='true')
disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=mapping.values())
disp.plot()
plt.show()

In [ ]:
print(sklearn.metrics.classification_report(yVal, yVal_pred, target_names=mapping.values()))

In [ ]:
f1  = sklearn.metrics.f1_score(     yVal, yVal_pred, average="macro")
auc = sklearn.metrics.roc_auc_score(yVal, yVal_prob, average="macro", multi_class='ovr')

print('f1-score: ', f1)
print('auc:      ', auc)